In [15]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [16]:
def crear_modelo_inicial(n_internas):
    model = Sequential()
    model.add(Dense(7, input_dim=7,activation='sigmoid'))
    for i in range(n_internas):
        model.add(Dense(14, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
    return model

def modificar_modelo(model_padre,alpha=0.1):
    pesos = model_padre.get_weights()
    l=[]
    for p in pesos:
        u = p + alpha*np.random.normal(size=p.shape)
        l.append(u)
    config = model_padre.get_config()
    new_model = Sequential.from_config(config)
    new_model.set_weights(l)
    new_model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
    return new_model
def actualizar_modelo(model_padre):
    pesos = model_padre.get_weights()
    config = model_padre.get_config()
    new_model = Sequential.from_config(config)
    new_model.set_weights(pesos)
    new_model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
    return new_model

In [17]:
def Scores(modelos):
    scores = []
    for model in modelos.model.tolist():
        model.fit(x_train, y_train,
              epochs=1000,
              batch_size=128,verbose = False)
        score = model.evaluate(x_test, y_test, batch_size=128,verbose=False)
        scores.append(score)
    return scores
        



In [18]:
titanic = pd.read_csv('train.csv')
titanic = titanic[['Survived','Pclass','Sex','Age','SibSp','Parch']]
titanic['target'] = titanic.Survived
titanic['Pclass_1']=np.where(titanic.Pclass==1,1,0)
titanic['Pclass_2']=np.where(titanic.Pclass==2,1,0)
titanic['Pclass_3']=np.where(titanic.Pclass==3,1,0)
titanic['Sex']=np.where(titanic.Sex=='male',1,0)
titanic = titanic.fillna(titanic.mean())
titanic[['Age']] = titanic[['Age']]/titanic[['Age']].mean()
del titanic['Pclass'],titanic['Survived']
len(titanic)

891

In [19]:
titanic_train = titanic.sample(600)
titanic_test = titanic.loc[~titanic.index.isin(titanic_train.index),:]
x_train = titanic_train[['Sex','Age','SibSp','Parch',
                         'Pclass_1','Pclass_2','Pclass_3']].values
y_train = titanic_train.target.values
x_test = titanic_test[['Sex','Age','SibSp','Parch',
                         'Pclass_1','Pclass_2','Pclass_3']].values
y_test = titanic_test.target.values

In [ ]:
## crear poblacion inicial

modelos = []
for n in range(10):
    for i in range(20):
        modelos.append([n,crear_modelo_inicial(n)])
modelos = pd.DataFrame(modelos)
modelos.columns =['n','model']


In [ ]:
for i in range(100):
    scores = Scores(modelos)
    u = pd.DataFrame(scores)
    modelos[['loss','score']] = u
    print(modelos.sort_values('score',ascending=False).head())
    modelos = modelos.sort_values('score',ascending=False)[0:int(len(modelos)/2)]
    modelos.model = modelos.model.apply(actualizar_modelo)
    l = modelos.model.apply(modificar_modelo)
    u =pd.DataFrame(list(zip(modelos.n.tolist(),l)))
    u.columns = ['n','model']
    modelos = modelos.append(u).reset_index(drop=True).fillna(0)


In [ ]:
modelos.head()


In [ ]:
best_model = modelos.iloc[0,1]




In [ ]:
titanic = pd.read_csv('test.csv')
titanic = titanic[['Pclass','Sex','Age','SibSp','Parch']]
titanic['Pclass_1']=np.where(titanic.Pclass==1,1,0)
titanic['Pclass_2']=np.where(titanic.Pclass==2,1,0)
titanic['Pclass_3']=np.where(titanic.Pclass==3,1,0)
titanic['Sex']=np.where(titanic.Sex=='male',1,0)
titanic = titanic.fillna(titanic.mean())
titanic[['Age']] = titanic[['Age']]/titanic[['Age']].mean()
del titanic['Pclass']







In [ ]:
resultado = pd.read_csv('test.csv')
resultado['Survived'] = best_model.predict_classes(titanic)
resultado[['PassengerId','Survived']].to_csv('result.csv',index=False)
